In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant
import pandas as pd

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")

def chat(user_message : str) -> dict[str, any]:
    # Generamos el resumen
    assistant.add_user_message(message=user_message)
    summary = assistant.get_memory_summary()
    print("summary")
    print(summary)
    
    # Generamos el requerimiento
    user_request = assistant.get_request(summary)
    print("user_request")
    print(user_request)
    
    # Clasificamos el requerimiento
    request_type = assistant.classify_request(user_request["request"])
    print("request_type")
    print(request_type)
    
    # Filtramos el requerimiento y obtenemos una respuesta
    first_response = assistant.filter_request(request_type["response"], user_request["request"], summary, user_message)
    print("first_response")
    print(first_response)
    
    # Procesamos la respuesta para entregar una respuesta final
    final_response = assistant.process_response(user_message, first_response["response"])
    print("final_response")
    print(final_response)
    
    # Actualizando el chat
    assistant.add_ai_message(message=final_response["final_answer"])
    
    response = {
        "summary" : summary,
        "request" : user_request,
        "request_type" : request_type,
        "response" : first_response["response"],
        "sql" : first_response["sql"],
        "dataframe" : first_response["dataframe"],
        "final_response" : final_response["final_answer"] 
    }
    
    return response

In [2]:
user_last_message = "Dame el promedio de la presion estatica del computador de flujo con tag FQI-EMED-3135.01-067 para los meses de agosto del 2023"
res = chat(user_last_message)

summary

        The human is asking for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.
user_request
{'request': 'The human is asking for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.'}
request_type
{'analysis': 'It seems to be a detailed request because the user is giving a tag for the flow computer, information parameter like static pressure and a time range.', 'response': 'complex-complete'}


C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)


first_response
{'sql': "SELECT AVG(vd.Valor) AS Average_Static_Pressure\nFROM dbo_v2.var_variable_datos vd\nJOIN dbo_v2.var_tipo_variable vt ON vd.idVariable_fk = vt.Id\nJOIN dbo_v2.fcs_computador_medidor cm ON vd.idSistemaMedicion_fk = cm.Id_Sisema_Medicion\nJOIN dbo_v2.fcs_computadores c ON cm.IdComputador_fk = c.Id\nWHERE c.Tag = 'FQI-EMED-3135.01-067'\nAND vt.Nombre = 'Pressão Estática (kPa)'\nAND MONTH(vd.Fecha) = 8\nAND YEAR(vd.Fecha) = 2023;", 'dataframe': [   Average_Static_Pressure
0                 -1.49857], 'response': "The average static pressure for the device with the Tag 'FQI-EMED-3135.01-067' in August 2023 is approximately -1.49857."}
final_response
{'final_answer': "El promedio de la presión estática para el dispositivo con la etiqueta 'FQI-EMED-3135.01-067' en agosto de 2023 es aproximadamente -1.49857."}


In [4]:
print(res["sql"])

SELECT AVG(vd.Valor) AS Average_Static_Pressure
FROM dbo_v2.var_variable_datos vd
JOIN dbo_v2.var_tipo_variable vt ON vd.idVariable_fk = vt.Id
JOIN dbo_v2.fcs_computador_medidor cm ON vd.idSistemaMedicion_fk = cm.Id_Sisema_Medicion
JOIN dbo_v2.fcs_computadores c ON cm.IdComputador_fk = c.Id
WHERE c.Tag = 'FQI-EMED-3135.01-067'
AND vt.Nombre = 'Pressão Estática (kPa)'
AND MONTH(vd.Fecha) = 8
AND YEAR(vd.Fecha) = 2023;


In [4]:
from langchain_core.messages import AIMessage

current_conversation = assistant.get_memory_summary()
current_messages = assistant._get_current_messages()

print("------------------ current_conversation ------------------")
print(current_conversation)
print("\n ------------------ current_messages ------------------")
for message in current_messages:
    if isinstance(message, AIMessage):
        print(f"AI Message: {message.content}")
    else:
        print(f"Human Message: {message.content}")

------------------ current_conversation ------------------

El humano está saludando a AI y le está pidiendo el promedio de la presión estática de un sistema de medición con la etiqueta 'FQI-EMED-3135.01-067' para los meses de agosto del 2023. AI responde que el promedio de la presión estática para ese sistema en agosto de 2023 es -1.49857 kPa.

 ------------------ current_messages ------------------
Human Message: Dame el promedio de la presion estatica del computador de flujo con tag FQI-EMED-3135.01-067 para los meses de agosto del 2023
AI Message: El promedio de la presión estática para el sistema con la etiqueta 'FQI-EMED-3135.01-067' en agosto de 2023 es -1.49857 kPa.
